In [3]:
def jaccard_score(img_name):
    intersection = 0.0
    union = 0.0
    
    ground_truth = Image.open('../../ISIC-2017_Training_Part1_GroundTruth/ISIC_' + img_name + '_segmentation.png')
    rgb_ground_truth = ground_truth.convert('RGB')
    
    output_im = Image.open("./grouped_clusters/" + img_name + ".png")
    rgb_output_im = output_im.convert('RGB')
    
    output_width, output_height = output_im.size
    truth_width, truth_height = ground_truth.size
    
    width_scale_const = float(truth_width)/float(output_width)
    height_scale_const = float(truth_height)/float(output_height)
    
    for y in range(output_height):
        for x in range(output_width):
            ground_pixel = rgb_ground_truth.getpixel((x*width_scale_const, y*height_scale_const))
            output_pixel = rgb_output_im.getpixel((x, y))
            
            isGround = ground_pixel == (255,255,255)
            isOutput = output_pixel == (255,255,255)
            
            if isGround and isOutput:
                intersection += 1.0
            if isGround or isOutput:
                union += 1.0
    
    print img_name + " - " + str(intersection/union)
    

In [8]:
from skimage.transform import rescale
from skimage.io import imread, imshow
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np

%matplotlib inline

# def group_clusters(img_name, isBlocked):
def group_clusters(img_name):
    path = '../../cmeans_noblock/ISIC_' + img_name + '_visual.gif'
    cluster_im = Image.open(path)
    original_im = Image.open('../../ISIC-2017_Training_Data/ISIC_' + img_name + '.jpg')
    rgb_cluster_im = cluster_im.convert('RGB')
    rgb_original_im = original_im.convert('RGB')

    scale_factor = float(cluster_im.size[0]) / float(original_im.size[0])
#     circle_pixels = remove_circle(img_name)
    
#     forbidden_clusters = set()
    clusters = {}
    try:
        #collect pixel values for each cluster
        width, height = cluster_im.size
        for y in range(height):
            for x in range(width):
                r, g, b = rgb_cluster_im.getpixel((x, y))
                cluster_id = str(r) + '-' + str(g) + '-' + str(b)
                pixel_id = str(x) + "-" + str(y)

#                 if pixel_id in circle_pixels:
#                     forbidden_clusters.add(cluster_id)

#                 if cluster_id not in forbidden_clusters:
                    original_rgb = rgb_original_im.getpixel((x/scale_factor, y/scale_factor)) #.3258 to deal with scale difference

                    if cluster_id in clusters:
                        clusters[cluster_id].append(original_rgb)
                    else:
                        clusters[cluster_id] = [original_rgb]

        print "# of clusters removed = " + str(len(forbidden_clusters))
        print "# of clusters = " + str(len(clusters))

        if len(clusters) <= 3:
            print "Exiting"
            return

        cluster_averages = {}

        #get average colour of clusters
        for cluster_id in clusters:
            if cluster_id not in forbidden_clusters:
                r, g, b = 0, 0, 0
                count = 0
                for pixel in clusters[cluster_id]:
                    r += pixel[0]
                    g += pixel[1]
                    b += pixel[2]
                    count += 1
                cluster_averages[cluster_id] = ((r/count), (g/count), (b/count))

        #Create Data for clustering
        list_of_pixels = []
        list_of_ids = {}
        counter = 0
        for cluster_id in cluster_averages:
            list_of_ids[cluster_id] = counter
            counter += 1
            list_of_pixels.append(list(cluster_averages[cluster_id]))

        cvecs = np.array(list_of_pixels)

        print 'Clustering...'

        #Cluster using k-means
        kmns = KMeans(n_clusters=2, random_state=1729)
        kmns.fit(cvecs)

        #Find darker of centroids
        cluster1 = kmns.cluster_centers_[0]
        cluster2 = kmns.cluster_centers_[1]

        avg1 = (cluster1[0] + cluster1[1] + cluster1[2])/3
        avg2 = (cluster2[0] + cluster2[1] + cluster2[2])/3

        #Generate output using clustered labels

        output = Image.new( 'RGB', (width,height), "black") # create a new black image
        output_pixels = output.load() # create the pixel map
        print "Creating output image"

        width, height = cluster_im.size
        for y in range(height):
            for x in range(width):
                r, g, b = rgb_cluster_im.getpixel((x, y))
                cluster_id = str(r) + '-' + str(g) + '-' + str(b)
                if cluster_id in list_of_ids:
                    index = list_of_ids[cluster_id]
                    if kmns.labels_[index] and avg2 < avg1:
                        output_pixels[x,y] = (255, 255, 255) # set the colour accordingly
                    elif not kmns.labels_[index] and avg1 < avg2:
                        output_pixels[x,y] = (255, 255, 255)

        print "Process Complete"
        plt.imshow(output)
        output.save("./grouped_clusters/" + img_name + ".png")
    except IndexError:
        print "error at ID # " + img_name

In [ ]:

isBlocked = True

#group_clusters('0000006')
# group_clusters('0000011')
# group_clusters('0000030')
# #group_clusters('0000036')
# group_clusters('0000045')
# group_clusters('0000169')
# group_clusters('0000047')
# group_clusters('0000125')
# group_clusters('0009564')
# group_clusters('0014582')


#for photo_number in range(0, 15295):    
for photo_number in range(294, 15295):    
    try:
        id = str(photo_number)
        new_id = ''

        num_digits = len(id)

        if num_digits < 7:
            difference = 7 - num_digits

            for digit in range(0, difference):
                new_id = new_id + '0'
            id = new_id + id

        group_clusters(id)
        jaccard_score(id)
        
    except IOError:
        print "error at ID # " + str(id)
        continue


# jaccard_score('0000000')
# jaccard_score('0000011')
# jaccard_score('0000030')



###########
Grouping clusters for 0000294
Removed circle
# of clusters removed = 1
# of clusters = 6
Clustering...
Creating output image
Process Complete
0000294 - 0.600160400674
###########
Grouping clusters for 0000295
Removed circle
# of clusters removed = 1
# of clusters = 6
Clustering...
Creating output image
Process Complete
0000295 - 0.516269470154
###########
Grouping clusters for 0000296
error at ID # 0000296
###########
Grouping clusters for 0000297
Removed circle
# of clusters removed = 1
# of clusters = 6
Clustering...
Creating output image
Process Complete
0000297 - 0.218428480379
###########
Grouping clusters for 0000298
Removed circle
# of clusters removed = 1
# of clusters = 6
Clustering...
Creating output image
Process Complete
0000298 - 0.416314143829
###########
Grouping clusters for 0000299
Removed circle
# of clusters removed = 2
# of clusters = 5
Clustering...
Creating output image
Process Complete
0000299 - 0.379041906615
###########
Grouping clusters for 0000300